In [2]:
tests = [{
  "query": "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в\n\n топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities",
  "id": 3
},
{
  "query": "В каком году Университет ИТМО был включён в число Национальных исследовательских университетов России?\n1. 2007\n2. 2009\n3. 2011",
  "id": 2
}
,
{
  "query": "В каком году Университет ИТМО был включён в число Национальных исследовательских университетов России?",
  "id": 2
}

]


for test in tests:
    print(contains_answer_variants(test))

test_vpn()

True
True
False
<html>
  <head>
    <style global>body{font-family:Arial,Helvetica,sans-serif}.container{align-items:center;display:flex;flex-direction:column;gap:2rem;height:100%;justify-content:center;width:100%}@keyframes enlarge-appear{0%{opacity:0;transform:scale(75%) rotate(-90deg)}to{opacity:1;transform:scale(100%) rotate(0deg)}}.logo{color:#8e8ea0}.scale-appear{animation:enlarge-appear .4s ease-out}@media (min-width:768px){.scale-appear{height:48px;width:48px}}.data:empty{display:none}.data{border-radius:5px;color:#8e8ea0;text-align:center}@media (prefers-color-scheme:dark){body{background-color:#343541}.logo{color:#acacbe}}</style>
  <meta http-equiv="refresh" content="390">
</head>
  <body>
    <div class="container">
      <div class="logo">
        <svg
          width="41"
          height="41"
          viewBox="0 0 41 41"
          fill="none"
          xmlns="http://www.w3.org/2000/svg"
          strokeWidth="2"
          class="scale-appear"
        >
          <path
 

In [1]:
from utils import contains_answer_variants, test_vpn
from dotenv import load_dotenv
load_dotenv() 


True

In [28]:
question_samples=[
    "как поступить в бакалавриат ИТМО в 2025 году?",
    "с какого года работает Медиагруппа «Мегабайт» итмо?",
    "какие были последние события в AI Learning Lab?",

                  ]

itmo_news = "https://news.itmo.ru/ru/"
itmo_abit = "https://abit.itmo.ru/"
university_live = "https://news.itmo.ru/ru/university_live/"
news_posts = [f"https://news.itmo.ru/ru/university_live/ratings/"]
itmo_urls = [itmo_news, itmo_abit, 
             "https://news.itmo.ru/ru/startups_and_business/" ]

In [4]:
from pprint import pprint
import bs4
import os


langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

## Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')

os.environ['LANGCHAIN_TRACING_V2'] = langchain_tracing_v2
os.environ['LANGCHAIN_ENDPOINT'] = langchain_endpoint
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

#Pinecone keys
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_API_HOST'] = pinecone_api_host
os.environ['PINECONE_INDEX_NAME'] = index_name




In [18]:

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from utils import parce_recurcuve
from pinecone import Pinecone
from langchain_pinecone import PineconeEmbeddings
import pickle

from bs4 import BeautifulSoup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from typing import List



In [31]:

docs_news_post = parce_recurcuve_depth(news_posts, max_depth=3)

In [173]:
# print(all_all_news)


docks_pkls = ["docs.pickle", "docs_all_all_news_smaller.pickle"]

with open(docks_pkls[0], 'rb') as f:
    docs_ = pickle.load(f)

with open(docks_pkls[1], 'rb') as f:
    docs_1 = pickle.load(f)

print(len(docs_))
print(len(docs_1))

3028
900


In [174]:
for doc in docs_1:
    if doc not in docs_:
        docs_.append(doc)

In [89]:
docs_all_all_news = load_documents_langchain_community(all_all_news)
import pickle
with open('docs_all_all_news_.pickle', 'wb') as handle:
    pickle.dump(docs_all_all_news, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [92]:
len(docs_all_all_news)

900

In [94]:
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['PINECONE_INDEX_NAME'])

# docs = parce_recurcuve_depth(itmo_urls, max_depth=2)

# docs = parce_recurcuve(itmo_urls)

# print(f"Total characters: {len(docs[0].page_content)}")
# len(docs) # 2268

In [13]:
import pickle
with open('docs.pickle', 'wb') as handle:
    pickle.dump(docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [46]:
import time 
from datetime import timedelta

def get_100_questions():
    template = "Ответь подробно и развернуто на каждый из этих вопросов (на русском языке):\n"
    questions = [
    "What is ITMO University known for?",
    "Where is ITMO University located?",
    "When was ITMO University founded?",
    "What are the main academic programs at ITMO University?",
    "Is ITMO University a public or private institution?",
    "How can international students apply to ITMO University?",
    "What are the admission requirements for ITMO University?",
    "Does ITMO University offer online courses?",
    "What are the tuition fees at ITMO University?",
    "Are there any scholarships available for students at ITMO University?",
    "What is the QS World Ranking of ITMO University?",
    "How does ITMO University support research activities?",
    "What are the main research areas at ITMO University?",
    "Does ITMO University have partnerships with international universities?",
    "What is the student-to-faculty ratio at ITMO University?",
    "Are there any exchange programs available at ITMO University?",
    "What are the main campus facilities at ITMO University?",
    "Where can students find accommodation near ITMO University?",
    "Does ITMO University provide dormitories for students?",
    "How many students study at ITMO University?",
    "What are the main student organizations at ITMO University?",
    "Does ITMO University have an entrepreneurship center?",
    "What is ITMO University’s role in technological innovation?",
    "Are there any IT-related competitions at ITMO University?",
    "How does ITMO University participate in international competitions?",
    "What programming languages are taught at ITMO University?",
    "Does ITMO University have a strong cybersecurity program?",
    "What is ITMO University’s reputation in artificial intelligence research?",
    "How does ITMO University support startups and innovations?",
    "Are there internship opportunities for ITMO students?",
    "What is the main language of instruction at ITMO University?",
    "Does ITMO University offer courses in English?",
    "What is the duration of undergraduate programs at ITMO University?",
    "What is the duration of master's programs at ITMO University?",
    "What is the duration of PhD programs at ITMO University?",
    "How competitive is admission to ITMO University?",
    "What are the most popular degree programs at ITMO University?",
    "How does ITMO University support international students?",
    "What is ITMO University's role in the Russian academic community?",
    "Does ITMO University have any Nobel laureates among its faculty or alumni?",
    "What kind of career opportunities do ITMO graduates have?",
    "Does ITMO University have a strong alumni network?",
    "Are there any famous alumni from ITMO University?",
    "Does ITMO University offer dual degree programs?",
    "How does ITMO University collaborate with the industry?",
    "What kind of scientific conferences does ITMO University host?",
    "What is the main library at ITMO University like?",
    "How can students access ITMO University’s digital resources?",
    "Does ITMO University offer short-term courses?",
    "What are the main fields of engineering taught at ITMO University?",
    "What sports facilities are available at ITMO University?",
    "Does ITMO University have a student exchange program with European universities?",
    "What kind of financial aid options does ITMO University provide?",
    "How many campuses does ITMO University have?",
    "What are the research laboratories at ITMO University focused on?",
    "How does ITMO University promote interdisciplinary studies?",
    "Does ITMO University participate in international rankings?",
    "What is the role of ITMO University in developing smart technologies?",
    "How does ITMO University contribute to artificial intelligence development?",
    "Are there any summer schools at ITMO University?",
    "What competitions in robotics are held at ITMO University?",
    "Does ITMO University have a robotics club?",
    "What are the main challenges of studying at ITMO University?",
    "How does ITMO University collaborate with high-tech companies?",
    "What career services does ITMO University offer?",
    "Does ITMO University have a career center?",
    "What is the cost of living for students in Saint Petersburg?",
    "Does ITMO University offer student mentoring programs?",
    "Are there any volunteer opportunities at ITMO University?",
    "How does ITMO University support academic publishing?",
    "What is the ITMO University’s approach to distance learning?",
    "Does ITMO University have a space research program?",
    "How does ITMO University promote scientific innovation?",
    "What kind of student projects are encouraged at ITMO University?",
    "Does ITMO University have a strong presence in AI research?",
    "What is ITMO University’s ranking in computer science?",
    "What are the admission deadlines for ITMO University?",
    "How does ITMO University support women in STEM?",
    "What are the key subjects in the computer science curriculum at ITMO University?",
    "Does ITMO University have an AI research lab?",
    "How does ITMO University handle student well-being?",
    "Are there any student hackathons at ITMO University?",
    "Does ITMO University provide free software licenses for students?",
    "What is ITMO University’s contribution to open-source projects?",
    "How is ITMO University connected to Russia’s tech industry?",
    "What are the major student events at ITMO University?",
    "How can students get involved in research at ITMO University?",
    "What is the employment rate for ITMO graduates?",
    "How can prospective students contact ITMO University?",
    "Does ITMO University have a cybersecurity research group?",
    "What role does ITMO University play in data science education?",
    "How does ITMO University incorporate modern technologies into education?",
    "What are ITMO University’s main academic partnerships?",
    "Does ITMO University have a nanotechnology program?",
    "How does ITMO University use AI in education?",
    "What is the ITMO Fellowship program?",
    "What languages can students learn at ITMO University?",
    "Are there any ITMO University alumni working in Silicon Valley?",
    "How does ITMO University rank compared to other Russian universities?",
    "What is ITMO University’s main focus in scientific research?",
    "Does ITMO University provide cloud computing resources for students?",
    "What is ITMO University’s approach to sustainable development?",
    "How can students engage in entrepreneurship at ITMO University?",
    "What kind of AI projects are developed at ITMO University?"
]

    return template, questions

In [ ]:
def dummy_pred(query):
    return ''

def test_100_questions():
    template, questions = get_100_questions()
    starttime = time.perf_counter()
    for i, each_q in enumerate(questions):
        answer = dummy_pred(each_q)
        print(f"each_q:\n{each_q}\nAnswer:\n{answer}")

    duration = timedelta(seconds=time.perf_counter()-starttime)
    print(f"\n{'*'*10}\nTotal : {duration}\nAvg : {duration / (i+1)}")

In [95]:
model_name = "multilingual-e5-large" # 

embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api_key

)


In [96]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pickle
from langchain_community.vectorstores import FAISS

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def split_docs(docs):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1020, chunk_overlap=10)
    splits = text_splitter.split_documents(docs)
    return splits


splits_all_news = split_docs(docs_all_all_news)

# vectorstore = Pinecone.from_documents(
#     documents=splits, 
#     embedding=embeddings,
#     #OpenAIEmbeddings(model="text-embedding-3-small"), 
#     # # text-embedding-3-large  multilingual-e5-large
#     index_name=index_name
# )


In [97]:
len(splits_all_news) # 39k

8101

In [84]:
from langchain_core.vectorstores.in_memory import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [98]:
# docsearch_all_news = FAISS.from_documents(splits_all_news, embeddings)
# vector_store.add_documents(documents=splits_all_news)

In [23]:
# faiss_index.index.ntotal # 25
# faiss_index.index.ntotal # 54 k
faiss_index.index.ntotal

31400

In [99]:
borders = list(range(0, len(splits_all_news)+1000, 1000))
for i in range(len(borders)-1):
    print(borders[i] , borders[i+1])

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000


In [100]:
import time

borders = list(range(0, len(splits_all_news)+1000, 1000))
for i in range(len(borders)-1):
    content = splits_all_news[borders[i] : borders[i+1]]
    # if i < 199: 
    #     pass
    # else:
    print(i)
    if i == 0:
        faiss_index = FAISS.from_documents(content,embeddings)
    else:
        time.sleep(61)
        faiss_index_i = FAISS.from_documents(content, embeddings)
        faiss_index.merge_from(faiss_index_i)


0
1
2
3
4
5
6
7
8


In [101]:
# faiss_index.save_local("faiss_index_all_news")

# faiss_index1 = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
# faiss_index.merge_from(faiss_index1)

In [177]:
# docsearch = FAISS.from_documents(splits, embeddings)
# docsearch.save_local("faiss_index1")
# faiss_index= FAISS.load_local("faiss_index1", embeddings, allow_dangerous_deserialization=True)


"faiss_index02"
"faiss_index1"
"faiss_index_all_news"

faiss_index = FAISS.load_local("faiss_index02", embeddings, allow_dangerous_deserialization=True)
docsearch_faiss_index1 = FAISS.load_local("faiss_index1", embeddings, allow_dangerous_deserialization=True)
docsearch_faiss_index_all_news = FAISS.load_local("faiss_index_all_news", embeddings, allow_dangerous_deserialization=True)

faiss_index.merge_from(docsearch_faiss_index1)
faiss_index.merge_from(docsearch_faiss_index_all_news)
faiss_index.save_local("faiss_index_all_summary")
# faiss_index1 = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
# faiss_index.merge_from(faiss_index1)

In [103]:

# retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

retriever = docsearch.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### Graph structure


In [108]:
!pip install langgraph


[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [197]:
def preprocess_user_q(q : str):
    template = "\nВыведи ответ в таком формате:\n1. {правильныя цифра ответа}\n2. {развернутое и подробное объяснение почему этот ответ правильный}\n"
    q = "Вот вопрос и варианты ответов:\n" + q
    q += template
    
    return q

def preprocess_user_q_no_answers(q : str):
    q = "Вот вопрос:\n" + q
    return q


In [201]:
import bs4
import re
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
# Define prompt for question-answering

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
# docsearch = 


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str



def retrieve(state: State):
    retrieved_docs = docsearch.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


def parce_links(source : str):
    pattern = '(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+'
    res = re.findall(pattern, source)
    return res if res else None




# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [143]:
# user_q = "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities"

user_q = "как работает AI-шлем, разработанный стартапом сотрудников ИТМО?\n1. решает проблему подготовки космонавтов\n2. решает проблему искуственного интеллекта а главный принцип работы это VR очки\n3. шлем похож на канадскую разработку RealWear. Ключевым элементом шлема является прозрачный дисплей дополненной реальности в виде очков."
test_q = preprocess_user_q(user_q)
print(test_q)

Вот вопрос и варианты ответов:
как работает AI-шлем, разработанный стартапом сотрудников ИТМО?
1. решает проблему подготовки космонавтов
2. решает проблему искуственного интеллекта а главный принцип работы это VR очки
3. шлем похож на канадскую разработку RealWear. Ключевым элементом шлема является прозрачный дисплей дополненной реальности в виде очков.
Выведи ответ в таком формате:
1. {правильныя цифра ответа}
2. {развернутое и подробное объяснение почему этот ответ правильный}



In [199]:
query_no_answers = "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?"

test_qno_answers = preprocess_user_q_no_answers(query_no_answers)
print(test_qno_answers)

Вот вопрос:
В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?


In [202]:
# result = graph.invoke({"question": test_q})
result = graph.invoke({"question": test_qno_answers})

In [207]:
query_no_answers = "В каком городе находится ИТМО?"

test_qno_answers = preprocess_user_q_no_answers(query_no_answers)
print(test_qno_answers)
result = graph.invoke({"question": test_qno_answers})
print(result)

Вот вопрос:
В каком городе находится ИТМО?
{'question': 'Вот вопрос:\nВ каком городе находится ИТМО?', 'context': [Document(id='f1100b79-3c69-4588-a981-c756f7aff68d', metadata={'source': 'https://news.itmo.ru/ru/education/students/news/13285/'}, page_content='Прочитайте также:\nКак поехать учиться в другую страну за счет государства. Опыт студента ИТМО, отправившегося в Китай\nИТМО ― снова в топ-100 университетов мира в предметном рейтинге QS по компьютерным наукам\n\n\n\nГлавный корпус ИТМО располагается на Петроградском острове, который входит в исторический центр Санкт-Петербурга. Фото: ITMO'), Document(id='eb4d0bb9-8db5-4eae-8be9-2c9899966705', metadata={'source': 'https://news.itmo.ru/ru/startups_and_business/business_success/news/9796/'}, page_content='Университет ИТМО'), Document(id='5267cf38-0e9b-4a46-87dc-c7fdec7a29e7', metadata={'source': 'https://news.itmo.ru/ru/university_live/ratings/news/9982/'}, page_content='Университет ИТМО'), Document(id='23f110c6-cad5-42aa-b3a5-1eddf

In [205]:
def process_answer_model(answer : str, sources : list):
    parts = answer.split('\n')
    id_answer = int(parts[0].split('1. ')[1])
    reasoning = parts[1].split('2. ')[1]
    links = list(set([doc.metadata['source'] for doc in sources]) )[:3]

    return id_answer, reasoning, links

def process_answer_model_no_answers(answer : str, sources : list):
    reasoning = answer
    id_answer = None
    links = list(set([doc.metadata['source'] for doc in sources]) )[:3]

    return id_answer, reasoning, links

process_answer_model_no_answers(result["answer"], result['context'])
id_answer, reasoning, links_raw_str = process_answer_model_no_answers(result["answer"], result['context'])
print('*'*20)
print(id_answer)
print(reasoning)
print(links_raw_str)

********************
None
ИТМО впервые вошёл в топ-400 мировых университетов по состоянию на 2021 год в рейтинге RUR Technical Sciences World University Ranking, заняв 41-е место. В этом рейтинге в 2021 году было представлено 790 ведущих мировых университетов в области технических наук.
['https://news.itmo.ru/ru/university_live/ratings/news/12778/', 'https://news.itmo.ru/ru/university_live/ratings/news/13236/', 'https://news.itmo.ru/ru/university_live/ratings/news/12551/']


In [189]:
from langgraph.graph import Graph
from fastapi import FastAPI
import uvicorn
# ! pip install langserve
from fastapi import FastAPI
from langserve import add_routes
from langchain_core.runnables import RunnableLambda

def inp(question: str) -> dict:
    question_ = preprocess_user_q(question)
    return {"question": list({question_})}


def out(value: dict):
    result = value
    return result


final_chain = RunnableLambda(inp) | graph | RunnableLambda(out)
test_q

'Вот вопрос и варианты ответов:\nВ каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities\nВыведи ответ в таком формате:\n1. {правильныя цифра ответа}\n2. {развернутое и подробное объяснение почему этот ответ правильный}\n'

In [190]:
final_chain.ainvoke({"question" : test_q})

<coroutine object RunnableSequence.ainvoke at 0x7f14cd26c440>

In [ ]:
async for output in rag_chain.astream(inputs, stream_mode="updates"):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][-1].pretty_print())
    print("\n---\n")

In [180]:


# user_q = "В каком городе находится главный кампус Университета ИТМО?\n1. Москва\n2. Санкт-Петербург\n3. Екатеринбург\n4. Нижний Новгород"
user_q = "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities"
test_q = preprocess_user_q(user_q)
res = rag_chain.ainvoke(test_q)
# pprint(res)

/tmp/ipykernel_32573/3333833504.py:4: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  res = rag_chain.ainvoke(test_q)


In [32]:
print(res)

1. 3  
2. Университет ИТМО впервые вошел в мировой ТОП-400 вузов в рейтинге QS World University Rankings, что подтверждается ростом его позиций в этом авторитетном рейтинге, публикуемом компанией Quacquarelli Symonds. Этот рейтинг считается одним из самых влиятельных в сфере высшего образования и привлекает внимание множества университетов по всему миру.  
3. [ИТМО](https://itmo.ru/en/news/10527/itmo_university_first_time_in_top_400_of_qs_world_university_rankings.htm)


In [34]:
# user_q = "В каком городе находится главный кампус Университета ИТМО?\n1. Москва\n2. Санкт-Петербург\n3. Екатеринбург\n4. Нижний Новгород"
user_q = "В каком рейтинге (по состоянию на 2021 год) ИТМО впервые вошёл в топ-400 мировых университетов?\n1. ARWU (Shanghai Ranking)\n2. Times Higher Education (THE) World University Rankings\n3. QS World University Rankings\n4. U.S. News & World Report Best Global Universities"
test_q = preprocess_user_q(user_q)
res = rag_chain.invoke(test_q)
pprint(res)

('1. 3  \n'
 '2. Университет ИТМО впервые вошел в мировой ТОП-400 вузов в рейтинге QS '
 'World University Rankings, что свидетельствует о его значительном росте и '
 'признании на международной арене. Этот рейтинг учитывает множество факторов, '
 'включая академическую репутацию, качество преподавания и исследования, что '
 'делает его одним из наиболее уважаемых в мире.  \n'
 '3. [Источник](https://itmo.ru)')


In [191]:
print(res)

<coroutine object RunnableSequence.ainvoke at 0x7f14c82f4240>


In [49]:
template = "\nВыведи ответ в таком формате:\n1. {правильныя цифра ответа}\n2. {объяснение почему этот ответ правильный}\n3. Сылки из интернет-источников\n"
print(template)


Выведи ответ в таком формате:
1. {правильныя цифра ответа}
2. {объяснение почему этот ответ правильный}
3. Сылки из интернет-источников



In [48]:
res

'2. Главный кампус Университета ИТМО находится в Санкт-Петербурге, Россия. Это подтверждается информацией, указанной в контексте.'

In [27]:
print(question_samples[0])
pprint(rag_chain.invoke(question_samples[0]))

как поступить в бакалавриат ИТМО в 2025 году?
('Чтобы поступить в бакалавриат ИТМО в 2025 году, необходимо ознакомиться с '
 'новыми правилами приема, которые включают возможность подачи согласия на '
 'зачисление без физического представления оригиналов документов. Также '
 'установлено единое количество направлений для выбора при поступлении. '
 'Рекомендуется следить за обновлениями на официальном сайте университета и '
 'готовиться к изменениям в процессе приема.')


In [28]:
print(question_samples[1])
pprint(rag_chain.invoke(question_samples[1]))

с какого года работает Медиагруппа «Мегабайт» итмо?
('Медиагруппа «Мегабайт» работает с 2014 года. Она включает в себя '
 'интернет-радио, интернет-газету и другие медиаформаты, ориентированные на '
 'студентов. Проект поддерживается университетами и правительством города.')


### Side hustles

In [23]:
# Documents
question = "How does LangChain handle multi-turn conversations in chat models?"
document = "LangChain supports chat models that manage complex, multi-turn conversations by maintaining state across interaction turns. This is achieved through structured message handling and conversation memory, which allows for the retention of historical context. Features like structured outputs and JSON-based response formatting further enable seamless integration with downstream applications, making it ideal for context-dependent use cases such as customer support and conversational AI."

model_name = "multilingual-e5-large" # 
from langchain_pinecone import PineconeEmbeddings
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api_key

)
document_result = embeddings.embed_query(document)


1024

In [28]:
splits = text_splitter.split_documents(docs)

In [29]:
splits

[]

In [27]:

docs_result = embeddings.embed_query(splits)


Exception: Invalid type for variable 'inputs'

In [24]:
len(document_result)

1024